In [1]:
import os
from dotenv import load_dotenv, find_dotenv

In [2]:
import nest_asyncio
nest_asyncio.apply()

import warnings
warnings.filterwarnings("ignore")

from llama_index.core import (
    PropertyGraphIndex,
    SimpleDirectoryReader,
    Settings,
    StorageContext,
    load_index_from_storage
)

from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core.indices.property_graph import (
    ImplicitPathExtractor,
    SimpleLLMPathExtractor
)
from llama_index.llms.groq import Groq
from llama_index.embeddings.ollama import OllamaEmbedding

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [3]:
Settings.llm = Groq("llama-3.3-70b-versatile")
Settings.embed_model=OllamaEmbedding(model_name="nomic-embed-text")
documents = SimpleDirectoryReader(input_files=["../docs/MBA_Worth_It.pdf"]).load_data(show_progress=True)

Loading files: 100%|██████████| 1/1 [00:00<00:00,  4.10file/s]


In [4]:
graph_store = Neo4jPropertyGraphStore(
    username="neo4j",
    password="llamaindex",
    url="bolt://localhost:7687",
)

In [5]:
from qdrant_client import QdrantClient, AsyncQdrantClient

client = QdrantClient(
    host="localhost",
    port=6333
)
aclient = AsyncQdrantClient(
    host="localhost",
    port=6333
)

vector_store = QdrantVectorStore(
    collection_name="mba",
    client=client,
    aclient=aclient,
    prefer_grpc=True,
)

In [6]:
index = PropertyGraphIndex.from_documents(
    documents,
    kg_extractors = [
        ImplicitPathExtractor(),
        SimpleLLMPathExtractor(
            num_workers = 4,
            max_paths_per_chunk = 10,
        )
    ],
    show_progress=True,
    property_graph_store=graph_store,
    vector_store=vector_store,
)

Parsing nodes:   0%|          | 0/7 [00:00<?, ?it/s]

Generating embeddings: 100%|██████████| 35/35 [00:03<00:00,  9.90it/s]


In [ ]:
query_engine = index.as_query_engine(
    include_text=True,
    similarity_top_k=8,
    graph_traversal_depth=2,
)

In [50]:
retriever = index.as_retriever(include_text=False)
nodes = retriever.retrieve("What are some considerations when deciding whether or not to embark on an MBA?")
for node in nodes:
    print(node.text)

Mba degree -> Offers -> Professional development
Mba -> Accelerates -> Professional development
Mba -> Is -> Graduate degree
Mba -> Is planned for -> Career advancement
Business school -> Offer -> Great time to gain exposure
Business school -> Provide -> Exposure to faculty
Business school -> Provide -> Exposure to peers
Some students -> Come to -> Business school
Business school -> Provides -> Academic benefits
Business school -> Provide -> Exposure to alumni network
Business school -> Requires -> Academic effort
Business school -> May not be -> Right step
Business school -> Offers -> Financial support


In [68]:
response = query_engine.query("What are some reasons to take an MBA?")

In [69]:
from IPython.display import display, Markdown

display(Markdown(str(response)))

One reason to take an MBA is for career advancement. It is also implied that having an MBA can help individuals get to where they want to be in their careers, as it is part of a larger plan to achieve their goals.

In [70]:
response = query_engine.query("What are some considerations when deciding whether or not to embark on an MBA?")

display(Markdown(str(response)))

Some considerations when deciding whether or not to embark on an MBA include reflecting on your current situation, thinking about where you are, where you want to go, and what it will take to get you there. It's also important to consider why you want to pursue an MBA, as it may not be the right step for everyone, especially if you're just looking to escape boredom or frustration in your current job. You should also think about what skills you need to acquire and whether an MBA will provide the necessary resources and leadership skills to help you achieve your career ambitions. Additionally, considering the various specializations and concentrations available in MBA programs can help you determine if it's the right fit for your career goals.

In [71]:
response = query_engine.query("What are the right motivations for an MBA?")

display(Markdown(str(response)))

The right motivations for an MBA include career advancement, professional development, and gaining access to more senior roles with higher salaries. It's also about figuring out what areas you want to focus on and making the most of your time to accelerate your career path. Additionally, having a clear understanding of where you are, where you want to go, and what it will take to get you there is crucial in making the right decision.

In [58]:
response = query_engine.query("What are the wrong motivations to take an MBA?")

display(Markdown(str(response)))

Empty Response

Why empty?!

In [59]:
response = query_engine.query("Why should anyone do an MBA?")

display(Markdown(str(response)))

An MBA offers several benefits, including a bigger paycheck, access to job opportunities, and a stronger professional network. It also helps individuals hone their skills, diversify their professional network, and make a career pivot. Additionally, an MBA provides access to connections and can accelerate professional development. Overall, it can be a valuable investment for career advancement.

In [60]:
response = query_engine.query("Why should anyone not do an MBA?")

display(Markdown(str(response)))

It can be costly, requiring a significant amount of money, and may also involve taking out student loans, as around 51% of MBA students do. Additionally, it takes time, typically a few years, and effort, which may be a deterrent for some individuals. The cost alone, although not the only deciding factor, can be a significant consideration. Furthermore, it requires proactivity and passion, which may not be suitable for everyone.

In [61]:
response = query_engine.query("So how should anyone decide on whether to do an MBA?")

display(Markdown(str(response)))

To decide on whether to do an MBA, one should consider where they are, where they want to go, and what it will take to get them there. They should think about their career goals and how an MBA can help them achieve those goals. It's also important to consider the cost and explore the various ways to pay for an MBA, such as fellowships, employer sponsorships, and student loans. Ultimately, the decision to pursue an MBA should be based on a careful consideration of the potential benefits and drawbacks, and whether it aligns with their overall career aspirations.

## Vector Store Index

In [62]:
from llama_index.core import VectorStoreIndex, StorageContext

vector_store2 = QdrantVectorStore(
    collection_name="mba_vectors_only",
    client=client,
    aclient=aclient,
    prefer_grpc=True,
)
storage_context = StorageContext.from_defaults(vector_store=vector_store2)
index2 = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context
)

In [65]:
query_engine2 = index2.as_query_engine(similarity_top_k=4)
response = query_engine2.query("What are the wrong motivations to take an MBA?")

display(Markdown(str(response)))

Cost alone should not be the deciding factor.

In [66]:
response = query_engine2.query("What are some considerations when deciding whether or not to embark on an MBA?")

display(Markdown(str(response)))

When deciding whether or not to embark on an MBA, some considerations include the cost of the program and how to finance it, as around 51% of students take out loans. However, cost alone should not be the deciding factor. Other considerations include the potential for fellowships and employer sponsorships to help pay for the program. Additionally, the potential career outcomes and the skills gained through the program, such as academic, interpersonal, and leadership skills, should be taken into account, as these can put graduates on an accelerated path into leadership positions and prepare them to solve complex business problems.